## 初期設定

In [ ]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
import hashlib

In [ ]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認します。


### データのロードの確認
ファイル重複やファイル欠損、破壊がないかを確かめます。
<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。
</details>

In [ ]:
print("")
print("ファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],[""]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("エラーなくロードできているようです。次に欠けてるファイルがないか確認します")
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    if df.empty:
        print(file,year,"データフレームが空です")
    else:
        #print("OK")
        continue

print("")
print("欠けているファイルはないようです、次に重複ファイルがないか確認します")
def df_hash(df, ignore_order=False):
    data_only = df.copy().fillna("").astype(str)
    if ignore_order:
        # 行順も無視（全列ソートしてから比較）
        data_only = data_only.sort_values(by=data_only.columns.tolist()).reset_index(drop=True)
    # 列名を無視して内容だけでハッシュを生成
    h = hashlib.md5(pd.util.hash_pandas_object(data_only, index=True).values)
    return h.hexdigest()
hash_map = defaultdict(list)
for (fname, year), df in df_DATAs_BY_ALL_FILEs.items():
    h = df_hash(df, ignore_order=False)  # ←行順を無視したいなら True に
    hash_map[h].append((fname, year))
duplicates = {h: keys for h, keys in hash_map.items() if len(keys) > 1}
if duplicates:
    print("🔍 同じ内容のデータフレーム（ヘッダー無視）が見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for fname, year in key_group:
            print(f"  - {fname}（{year}）")
        print()
else:
    print("✅ 重複データフレームはありません。")

print("")
print("重複しているファイルはないようです。")
print("")
print("データロードの確認を終了します。")


### 欠損値表現の確認と処理
欠損値の表現を確認します。  
setting.yamlのnan_valuesに登録します
<details>
<summary><b>結果</b></summary>

</details>

In [ ]:
# 欠損値の表現確認
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],[""]
)
#display(df_DATAs_BY_ALL_FILEs)
print("")
print("ファイルごとに各文字列の個数をカウントします")

COUNT_COLUMNS = ["col", "nan", "na", "n/a", "-", "--", "none", "0", "space", "empty", "alphabet"] 
FINAL_COLUMNS = COUNT_COLUMNS + ["ファイル名", "会計年"] # 追跡を明確にするため
df_placeholder_counts = pd.DataFrame(columns=FINAL_COLUMNS)
current_index = 0
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    for col in df:
        ser = df[col].fillna("")
        ser_str = ser.astype(str)
        #display(ser_str)
        row_data_tuple = (
            col,
            (ser_str == 'nan').sum(),
            (ser_str == 'na').sum(),
            (ser_str == 'n/a').sum(),
            (ser_str == '-').sum(),
            (ser_str == '--').sum(),
            (ser_str == 'none').sum(),
            (ser_str == '0').sum(),
            (ser_str == ' ').sum(),
            (ser_str == '').sum(),
            ser_str.str.contains('[A-Za-z]', na=False).sum()
        )
        df_placeholder_counts.loc[current_index, COUNT_COLUMNS] = row_data_tuple
        df_placeholder_counts.loc[current_index, "ファイル名"] = filename
        df_placeholder_counts.loc[current_index, "会計年"] = year

        current_index += 1
#display(df_placeholder_counts)


In [173]:
print("")
print("欠損値と疑われるコードを出します")
df = df_placeholder_counts.drop(["ファイル名","会計年"],axis=1)
df = df.groupby("col").sum().sum()
#display(df)

print("")
print("ハイフンとゼロが該当しそうなことがわかりました。")
print("0が登録されているファイルと列をを検索します。")
df = df_placeholder_counts
df = df.groupby(["ファイル名","col"]).sum()["0"]
df = df[df>0].reset_index()
display(df)
print("")
print("4つのファイルごとに該当列のリストを準備します")
col_list_series = df.groupby('ファイル名')['col'].apply(list)
col_lists = col_list_series.to_dict()
#print(col_lists)

print("")
print("まずは年でわかれた各ファイルをつなぎ合わせます")
df_merge = {}
for (filename, year), df in df_DATAs_BY_ALL_FILEs.items():
    if filename not in df_merge:
        df_merge[filename] = []
    df_merge[filename].append(df)
df_final = {}
for filename in df_merge:
    df_final[filename]  = pd.concat(df_merge[filename])
#display(df_final["fy-balance-sheet.csv"])

print("")
print("ファイルごとにゼロ値周りをチェックします")
filename = "fy-cash-flow-statement.csv"
df_all = df_final[filename]
for col_value in col_lists[filename]:
    #print(filename, col)
    df = df_all[df_all[col_value] == "0"]
    code_list = df["コード"].unique()
    print("-----------------------",col_value,"-----------------------")
    for code in code_list:
        display(df_all[df_all["コード"] == code])


欠損値と疑われるコードを出します

ハイフンとゼロが該当しそうなことがわかりました。
0が登録されているファイルと列をを検索します。


,ファイル名,col,0
0,fy-balance-sheet.csv,BPS,5
1,fy-balance-sheet.csv,短期借入金,12
2,fy-balance-sheet.csv,総資産,1
3,fy-balance-sheet.csv,自己資本比率,1
4,fy-balance-sheet.csv,長期借入金,17
5,fy-cash-flow-statement.csv,営業CF,2
6,fy-cash-flow-statement.csv,営業CFマージン,21
7,fy-cash-flow-statement.csv,投資CF,15
8,fy-cash-flow-statement.csv,現金同等物,2
9,fy-cash-flow-statement.csv,設備投資,234



4つのファイルごとに該当列のリストを準備します

まずは年でわかれた各ファイルをつなぎ合わせます

ファイルごとにゼロ値周りをチェックします
----------------------- 営業CF -----------------------


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
2236,7544,2010-02-01,969000000,-788000000,388000000,-714000000,1617000000,3.66
2306,7544,2011-02-01,1289000000,-481000000,-1664000000,-396000000,760000000,4.78
2384,7544,2012-02-01,2127000000,-583000000,-703000000,-420000000,1601000000,7.9
2477,7544,2013-02-01,0,-638000000,-394000000,-642000000,568000000,0
2558,7544,2014-02-01,924000000,-227000000,-481000000,-380000000,785000000,4.12
2645,7544,2015-02-01,1435000000,-228000000,-691000000,-449000000,1300000000,6.84
2727,7544,2016-02-01,453000000,-579000000,-323000000,-722000000,850000000,2.38
2829,7544,2017-02-01,-2634000000,3614000000,775000000,-112000000,2605000000,-16.06
2918,7544,2018-02-01,-6081000000,10113000000,-1774000000,-244000000,4863000000,-48.36
2988,7544,2019-02-01,-1087000000,78000000,-241000000,-101000000,3612000000,-9.06


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
762,3606,2010-02-01,2273000000,-5151000000,-7901000000,-896000000,9700000000,1.76
782,3606,2011-02-01,-222000000,935000000,2202000000,-333000000,12570000000,-0.3
811,3606,2012-02-01,-1879000000,-877000000,-25000000,-571000000,9781000000,-2.52
835,3606,2013-02-01,243000000,45000000,-13000000,-720000000,10096000000,0.32
860,3606,2014-02-01,0,-98000000,2800000000,-1065000000,12909000000,0
878,3606,2015-02-01,741000000,-298000000,-70000000,-500000000,13320000000,1.03
888,3606,2016-02-01,-4201000000,-39000000,-83000000,-345000000,8960000000,-5.9
897,3606,2017-02-01,-256000000,-178000000,-88000000,-412000000,8404000000,-0.38
904,3606,2018-02-01,1547000000,-4195000000,3406000000,-3927000000,9187000000,2.33
909,3606,2019-12-01,-4567000000,1090000000,-1025000000,-253000000,3316000000,-9.09


...

,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
3171,9836,2010-11-01,-1430000000,5011000000,-86000000,-145000000,3660000000,-10.86
3249,9836,2011-11-01,-1231000000,-337000000,-87000000,-78000000,2002000000,-13.39
3336,9836,2012-11-01,578000000,-31000000,-72000000,-24000000,2476000000,6.01
3430,9836,2013-11-01,-632000000,-30000000,-40000000,-42000000,1772000000,-6.75
3525,9836,2014-11-01,-1261000000,74000000,0,-71000000,527000000,-11.67
3623,9836,2015-11-01,-33000000,-85000000,324000000,-51000000,732000000,-0.28
3720,9836,2016-11-01,-107000000,224000000,-78000000,-48000000,771000000,-0.87
3833,9836,2017-11-01,838000000,-285000000,-248000000,-197000000,1075000000,6.44
3950,9836,2018-11-01,98000000,-232000000,-14000000,-179000000,926000000,0.68


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
2921,9078,2010-03-01,2552000000,-508000000,-1787000000,-947000000,2453000000,6.58
2995,9078,2011-03-01,1770000000,-1392000000,-262000000,-1422000000,2568000000,4.28
3078,9078,2012-03-01,1015000000,-1353000000,-506000000,-1473000000,1724000000,2.51
3172,9078,2013-03-01,1917000000,-1054000000,-571000000,-966000000,2015000000,4.78
3260,9078,2014-03-01,2137000000,-1500000000,89000000,-1499000000,2741000000,4.94
3350,9078,2015-03-01,2424000000,-1740000000,0,-2587000000,3425000000,5.47
3444,9078,2016-03-01,2232000000,-1698000000,-527000000,-1898000000,3051000000,5.04
3554,9078,2017-03-01,2021000000,-2114000000,4000000,-2293000000,2963000000,4.54
3645,9078,2018-03-01,2773000000,-1377000000,-350000000,-1404000000,4009000000,5.92
3716,9078,2019-03-01,2600000000,-3566000000,1388000000,-3790000000,4432000000,5.29


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
899,3858,2010-03-01,469525000,-430000000,-60074000,-94396000,418280000,40.49
932,3858,2011-03-01,457890000,558013000,57839000,-104790000,1492023000,32.19
963,3858,2012-03-01,-49512000,-393939000,-76515000,-148361000,972056000,-5.41
987,3858,2013-03-01,214766000,-829140000,110185000,-232842000,467868000,24.07
1012,3858,2014-03-01,-178906000,177749000,71275000,-72918000,537986000,-23.4
1032,3858,2015-03-01,-170152000,32719000,296607000,-32292000,697161000,-19.21
1042,3858,2016-03-01,-124076000,-593364000,1384230000,-62219000,1363950000,-12.79
1051,3858,2017-03-01,94210000,622788000,-48813000,-20178000,2029002000,8.38
1058,3858,2018-03-01,10366000,-1089960000,5908000,-27154000,954957000,0.44
1062,3858,2019-03-01,118309000,70458000,15655000,-81406000,1159343000,4.92


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
1807,5137,2019-09-01,-,-,-,-,945983000,-
1806,5137,2020-09-01,-665258000,-34873000,207700000,-51496000,454051000,-166.92
1836,5137,2021-09-01,-268266000,3333000,1116985000,-4600000,1307149000,-32.41
1821,5137,2022-09-01,-445965000,-16591000,0,-15591000,844922000,-35.54
1798,5137,2023-09-01,-107325000,-113325000,347243000,-113583000,971167000,-6.28
1752,5137,2024-09-01,46024000,-231481000,23588000,-201616000,807170000,2.12
1840,5137,2024-09-01,46024000,-231481000,23588000,-201616000,807170000,2.12


,コード,年度,営業CF,投資CF,財務CF,設備投資,現金同等物,営業CFマージン
1740,6047,2013-05-01,-29330000,-2430000,74577000,-430000,42967000,-7033.57
1793,6047,2014-05-01,-1512000000,-20000000,1952000000,-6952000,462000000,-421.17
1844,6047,2015-05-01,361000000,-164000000,5650000000,-46522000,6311000000,11.41
1891,6047,2016-05-01,532000000,-1275000000,-191000000,-13395000,5369000000,11.57
1955,6047,2017-05-01,1698000000,-562000000,70000000,-116000000,6576000000,21.94
2014,6047,2018-05-01,2392000000,-329000000,127000000,-24000000,8767000000,21.36
2073,6047,2019-05-01,2067000000,130000000,192000000,-24000000,11157000000,13.76
2116,6047,2020-05-01,-631000000,-986000000,-459000000,-623000000,9075000000,-4.51
2157,6047,2021-05-01,983000000,-1079000000,-1000000,-66000000,8952000000,11.03
2141,6047,2022-05-01,-33000000,-4064000000,9000000,-15000000,4947000000,-0.37


### データ型の整合性確認

<details>
<summary><b>結果</b></summary>

</details>

### 初期品質の確認

<details>
<summary><b>結果</b></summary>

</details>


In [ ]:
# 財務データ 結合前
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

def get_file_info(df_datas_by_all_files):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for file, year in df_datas_by_all_files.keys():
        df = df_datas_by_all_files[(file,year)]
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            file, year, df["コード"].nunique(), df["年度"].nunique(),df.columns.nunique()
        ]
    return df_file_info

print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],["-","","0"]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
df_file_info = get_file_info(df_DATAs_BY_ALL_FILEs)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
"""fig = px.line(df_file_info,x="year",y="code_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
fig.show()"""

print("")
print("列数はOKです")
print("最新年のファイルに登録されている年度を表示します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年のファイルに過去のデータがあります。")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"])

print("")
print("コードを限定し同じかどうか調べます。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("違っています。これは更新データと思われます。")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
df_DATAs_BY_ALL_FILEs = dl.update_duplicated(df_DATAs_BY_ALL_FILEs, 2025)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_DATAs_BY_ALL_FILEs.keys():
    if year == 2025:
        df = df_DATAs_BY_ALL_FILEs[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_DATAs_BY_ALL_FILEs[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_DATAs_BY_ALL_FILEs)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).sort_index().reset_index()
#display(df_compare)
fig = px.line(df_compare,x="year",y="code_counts",color="区分")
fig.show()


In [ ]:
# 財務データ ファイルごとの結合
df_DATAs_BY_FILEs = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"],)
target_file = settings["files"][0]
display(df_DATAs_BY_FILEs[target_file][df_DATAs_BY_FILEs[target_file]["コード"]=="130A"])

In [ ]:
# 財務データ　全結合
df_ALL_DATAs = dl.merge_all_data(df_DATAs_BY_FILEs)
print(df_ALL_DATAs.shape)
#display(df_ALL_DATAs)
display(df_ALL_DATAs["コード"].unique())
display(df_ALL_DATAs["年度"].nunique())

In [ ]:
# 上場企業の情報
df = dl.load_code_list_info(settings["data_path"], settings["files_reference"][0])
print(settings["files_reference"][0], df.shape)
display(df)

### データ型の整合性チェック

### 初期品質の概要把握